# Visualizing exclusion contours

Having [already executed][inference README] the jobs with funcX to calculate the observed and expected $\mathrm{CL}_s$ values for the mass hypotheses for the [electroweakino $1\ell bb$ search](https://www.hepdata.net/record/ins1755298) let's go beyond just performing the calculations and quickly visualize the exclusion contours for the analysis.

[inference README]: https://github.com/iris-hep/analysis-grand-challenge/blob/main/workshops/agctools2022/statistical-inference/README.md

In [ ]:
%matplotlib widget

import array
import json
import pathlib

import matplotlib.lines
import matplotlib.pyplot as plt
import matplotlib.transforms
import mplhep
import numpy as np
import requests
import uproot

In [ ]:
# Library made for this workshop (not on PyPI)
from exclusion.visualize import plot_contour

In [ ]:
with open("results.json") as read_file:
    results = json.load(read_file)

Create some visualization helper functions for the styling of the plots

In [ ]:
def atlas_label(ax, suffix=None, lumi_label=None):
    if suffix is None:
        suffix = "Summary"
    if lumi_label is None:
        lumi_lable = ""

    text, suffix = mplhep.atlas.text(ax=ax, loc=2, text=suffix)
    fig = ax.get_figure()
    bbox = text.get_window_extent(renderer=fig.canvas.get_renderer())
    bbox_axes = matplotlib.transforms.Bbox(
        suffix.get_transform().inverted().transform(bbox)
    )

    label = mplhep.label.ExpSuffix(
        *suffix.get_position(),
        text=lumi_label,
        transform=suffix.get_transform(),
        ha=suffix.get_ha(),
        va=suffix.get_va(),
        fontsize=suffix.get_fontsize(),
        fontname=suffix.get_fontname(),
        fontstyle="normal"
    )
    ax._add_text(label)
    suffix.set_position(
        (
            text.get_position()[0] + bbox_axes.width + 0.01,
            text.get_position()[1] + bbox_axes.height,
        )
    )
    suffix.set_fontsize(text.get_fontsize())

In [ ]:
def kinematic_exclusion(ax):
    line = ax.axline((150, 25), (650, 525), linestyle="-.", color="#cccccc", alpha=0.9)
    p1 = ax.transData.transform_point((150, 25))
    p2 = ax.transData.transform_point((650, 525))
    dy = p2[1] - p1[1]
    dx = p2[0] - p1[0]
    rotation = np.degrees(np.arctan2(dy, dx))
    ax.text(
        200,
        100,
        r"$m(\tilde{\chi}^{\pm}_{1}/\tilde{\chi}^{0}_{2}) < m(\tilde{\chi}^{0}_{1}) + 125\ \mathrm{GeV}$",
        va="baseline",
        fontsize="small",
        color="#cccccc",
        alpha=0.9,
        rotation=rotation,
    )

In [ ]:
# Use ATLAS style for the plots
plt.style.use(mplhep.style.ATLAS)

`exclusion.visualize.plot_contour` will perform interpolation between the mass hypothesis points using SciPy (with a chosen default configuration) and then plot those contours.

Note that at the moment `exclusion.visualize` does not impliment kimenatic cutoffs on the contours, and so one should mentally adjust the interpolation to remove any component above the cutoff line $m(\tilde{\chi}^{\pm}_{1}/\tilde{\chi}^{0}_{2}) < m(\tilde{\chi}^{0}_{1}) + 125\ \mathrm{GeV}$

In [ ]:
def plot_exclusions(ax=None):
    if ax is None:
        fig, ax = plt.subplots()

    plot_contour(
        ax,
        results,
        label="Open Likelihood",
        color="steelblue",
        show_points=True,
        show_interpolated=True,
    )

    lumi_label = (
        r"$\sqrt{s} = \mathrm{13\ TeV}, 139\ \mathrm{fb}^{-1}$"
        + "\nAll limits at the 95% CL"
    )
    atlas_label(ax, suffix="Open Likelihood", lumi_label=lumi_label)

    # Set plot ranges
    mass_ranges = np.asarray(
        [values["mass_hypotheses"] for _, values in results.items()]
    ).T

    ax.set_xlim(mass_ranges[0].min(), mass_ranges[0].max() + 100)
    ax.set_ylim(
        mass_ranges[1].min() if mass_ranges[1].min() > 0 else 0,
        mass_ranges[1].max() + 100,
    )

    # To get angle correct need to run after bounds of plot are finalized
    kinematic_exclusion(ax)

    # ax.legend(loc="upper right")
    ax.legend(loc=(0.05, 0.6))

    # add process label
    process_label = r"$\tilde{\chi}^{0}_{2}\tilde{\chi}^{\pm}_{1} \rightarrow Wh\ \tilde{\chi}^{0}_{1}\tilde{\chi}^{0}_{1}$"
    ax.text(0.0, 1.01, process_label, transform=ax.transAxes, va="bottom")

    ax.set_xlabel(r"$m(\tilde{\chi}_{1}^{\pm}/\tilde{\chi}_{2}^{0})$ [GeV]")
    ax.set_ylabel(r"$m(\tilde{\chi}_{1}^{0})$ [GeV]")

    return ax

In [ ]:
ax = plot_exclusions();

(Get the figure dimensions for later use)

In [ ]:
bbox = ax.get_window_extent().transformed(ax.get_figure().dpi_scale_trans.inverted())
fig_width, fig_height = bbox.width, bbox.height

## Compare against published TGraphs on HEPData

c.f. https://gist.github.com/kratsg/4ff8cb2ded3b25552ff2f51cd6b854dc

In [ ]:
analyses = {
    "1Lbb": {
        "hepdata": "ins1755298",
        "color": "#9394db",
        "exp": "https://www.hepdata.net/download/table/ins1755298/Expected%20limit%201lbb/3/root",
        "obs": "https://www.hepdata.net/download/table/ins1755298/Observed%20limit%201lbb/3/root",
    }
}

In [ ]:
def get_filename(analysis, details, kind):
    """
    For a given analysis name and details on where the expected/observed curves are located,
    download the corresponding kind of curve locally and cache it at data/{analysis}/{kind}.root.

    Args:
        analysis (str): analysis name (the key in the analyses object above)
        details (dict): analysis details (the value in the analyses object above)
        kind (str): specify either 'exp' or 'obs', according to the details provided

    Returns:
        file path (pathlib.Path): The local ROOT file
    """
    assert kind in ["exp", "obs"], f"'{kind}' must be either 'exp' or 'obs'"

    if not details[kind]:  # skip empty ones
        return None

    analysis = "".join([c for c in analysis if c.isalpha() or c.isdigit()]).rstrip()

    folder = pathlib.Path("data").joinpath(details["hepdata"]).joinpath(analysis)
    fpath = folder.joinpath(f"{kind}.root")
    if not fpath.is_file():
        fpath.parent.mkdir(parents=True, exist_ok=True)
        response = requests.get(details[kind])
        response.raise_for_status()
        fpath.write_bytes(response.content)
    return fpath

In [ ]:
for analysis, details in analyses.items():
    for kind in ["exp", "obs"]:
        print(get_filename(analysis, details, kind))

In [ ]:
def get_graph(root_file):
    it = iter(k for k, v in root_file.classnames().items() if v not in ["TDirectory"])
    return root_file[next(it)]

In [ ]:
for analysis, details in analyses.items():
    for kind in ["exp", "obs"]:
        fname = get_filename(analysis, details, kind)
        if not fname:
            continue
        with uproot.open(fname) as f:
            print(get_graph(f))

In [ ]:
def plot_graphs(ax=None, suffix=None):
    if ax is None:
        fig, ax = plt.subplots()

    # Axes
    ax.set_xlim([150, 1100])
    ax.set_ylim([0, 525])

    # add process label
    process_label = r"$\tilde{\chi}^{0}_{2}\tilde{\chi}^{\pm}_{1} \rightarrow Wh\ \tilde{\chi}^{0}_{1}\tilde{\chi}^{0}_{1}$"
    ax.text(0.0, 1.01, process_label, transform=ax.transAxes, va="bottom")

    # Set up initial legend
    leg1_elements = [
        matplotlib.lines.Line2D(
            [0], [0], linestyle="--", color="black", label="Expected"
        ),
        matplotlib.lines.Line2D(
            [0], [0], linestyle="-", color="black", label="Observed"
        ),
    ]
    leg1 = ax.legend(title="All limits at 95% CL", handles=leg1_elements)

    # Create legend for the analyses added
    leg2_elements = []

    # ATLAS Labeling
    lumi_label = r"$\sqrt{s} = \mathrm{13\ TeV}, 139\ \mathrm{fb}^{-1}$"
    atlas_label(ax, suffix=suffix, lumi_label=lumi_label)

    # get analysis curves
    for analysis, details in analyses.items():
        leg2_elements.append(
            matplotlib.lines.Line2D(
                [0], [0], linestyle="-", color=details["color"], label=analysis
            )
        )

        f_exp_name = get_filename(analysis, details, "exp")
        f_obs_name = get_filename(analysis, details, "obs")

        if f_exp_name:
            f_exp = uproot.open(get_filename(analysis, details, "exp"))
            graph_exp = get_graph(f_exp)
            ax.plot(
                graph_exp.member("fX"),
                graph_exp.member("fY"),
                linestyle="--",
                color=details["color"],
                alpha=0.9,
            )

        if f_obs_name:
            f_obs = uproot.open(get_filename(analysis, details, "obs"))
            graph_obs = get_graph(f_obs)
            ax.plot(
                graph_obs.member("fX"),
                graph_obs.member("fY"),
                linestyle="-",
                color=details["color"],
                alpha=0.9,
                label=analysis,
            )

    # Draw Lines
    kinematic_exclusion(ax)

    # Axis Labels
    ax.set_xlabel(r"$m(\tilde{\chi}^{\pm}_{1}/\tilde{\chi}^{0}_{2})\ \mathrm{[GeV]}$")
    ax.set_ylabel(r"$m(\tilde{\chi}^{0}_{1})\ \mathrm{[GeV]}$")

    # for multiple legends
    ax.legend(
        loc="upper left",
        bbox_to_anchor=(0.01, 0.85),
        handles=leg2_elements,
        fontsize="small",
    )
    ax.add_artist(leg1)

    return ax

In [ ]:
plot_graphs();

In [ ]:
# Show overlay
ax = plot_graphs(suffix="Open Likelihood")
plot_contour(
    ax,
    results,
    label="Open Likelihood",
    color="steelblue",
    show_interpolated=True,
)
ax.legend(loc=(0.05, 0.6));

When comapred with [the published plot](https://ar5iv.labs.arxiv.org/html/1909.09226#S8.F6) things look reasonable and any differences can be explaiend by interpolation choices given the mass points grid

In [ ]:
# Exploit GitHub CDN ![arXiv_figure_6](https://user-images.githubusercontent.com/5142394/165033580-ed1104f3-5548-47af-8226-6d9469a79267.png)
! mkdir -p figures
! curl -sL https://user-images.githubusercontent.com/5142394/165033580-ed1104f3-5548-47af-8226-6d9469a79267.png --output figures/arxiv_1909.09226_figure_6.png

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(fig_width * 2.2, fig_height * 1.3))
plot_exclusions(ax1)
ax1.legend(loc=(0.05, 0.6))
img = plt.imread("figures/arxiv_1909.09226_figure_6.png")
ax2.imshow(img, aspect="auto")
ax2.axis("off")
fig.tight_layout()